In [1]:
import numpy as np
import scipy as sp
import skimage.io
import skimage.viewer
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score
import ipympl
import os

%matplotlib widget

def load_image(path, smooth=False, labels_to_smooth=None):
    """Loads an image (or image stack) given a defined path.
    
    Args:
        path(string): String containing path for images with proper file extension. 
            Example: "images/*.tif"
    
    Returns: 
        numpy.ndarray: Array with images defined in path.
    """

    seq_img = skimage.io.imread_collection(load_pattern=path, conserve_memory=True)
    print(seq_img)
    img_list = []
    num_img = 0 
    for image in seq_img:
        if smooth: # Todo: Verify error when labels_to_smooth=None
            smooth_img = smooth_labels(image=seq_img[num_img], labels=labels_to_smooth) 
            img_list.append(smooth_img)
        else:
            img_list.append(seq_img[num_img])

        num_img = num_img + 1 # After for loop, num_img will represent the stack size
    loaded_img = np.array(img_list)
    
    return loaded_img

def filter_labels(image, labels):
    """From a segmented image, returns an image with the specified labels.
    
    Args:
        image(numpy.ndarray): Array defining an image (or image stack). 
        labels(list[int]): List containing labels to be filtered. 
  
    Returns: 
        numpy.ndarray: Array with filtered image.
    """
    
    filtered_img = np.zeros((image.shape), dtype=np.int8) # Todo: Verify with just one image
    for label in labels:
        selection = image.copy()
        mask = (image == label)
        selection[~mask] = 0
        filtered_img[selection == label] = label
    
    return filtered_img  

def smooth_labels(image, labels):
    """Applies median filter to certain labels as a post-processing tool.
    
    Args:
        image(numpy.ndarray): Array defining an image (or image stack). 
        labels(list[int]): List of integers defining the labels to be smoothed.
    
    Returns: 
        numpy.ndarray: Array with post-processed image.
    """
    
    filtered_labels = filter_labels(image=image, labels=labels)
    median = skimage.filters.median(filtered_labels)
    for label in labels:
        image[median == label] = label
    
    return image

def count_voxels_per_label(image, label):
    """Counts the number of voxels of a determined label from an image.
    
    Args:
        image(numpy.ndarray): Array representing image (or image stack). 
        label(int): Label value
    
    Returns: 
        int: Number of voxels of defined label
    """
    
    histogram = skimage.exposure.histogram(image)
    label_index = np.where(histogram[0] == label)[0]
    number_of_voxels = histogram[0][label]
    
    return number_of_voxels

def contact_area(image, label1, label2):
    """Counts the number of surfaces of label1 in contact with label2.
    
    Args:
        image(numpy.ndarray): Array representing image (or image stack). 
        label1(int): Value of label 1
        label1(int): Value of label 2
  
    Returns: 
        int: Number of voxels of label1 in contact with label2
    """
    
    histogram = skimage.exposure.histogram(image)
    
    if label1 not in histogram[1] or label2 not in histogram[1]:
        raise ValueError('One or more labels do not exist. Please input valid labels.') 
        
    x_contact_1 = np.logical_and(image[:, :, :-1] == label1, image[:, :, 1:] == label2)
    x_contact_2 = np.logical_and(image[:, :, :-1] == label2, image[:, :, 1:] == label1)
    y_contact_1 = np.logical_and(image[:, :-1, :] == label1, image[:, 1:, :] == label2)
    y_contact_2 = np.logical_and(image[:, :-1, :] == label2, image[:, 1:, :] == label1)
    z_contact_1 = np.logical_and(image[:-1, :, :] == label1, image[1:, :, :] == label2)
    z_contact_2 = np.logical_and(image[:-1, :, :] == label2, image[1:, :, :] == label1)
    #np.argwhere(hpairs) - counts each pair which is in contact

    contact_voxels = np.count_nonzero(x_contact_1) + np.count_nonzero(x_contact_2) \
                     + np.count_nonzero(y_contact_1) + np.count_nonzero(y_contact_2) \
                     + np.count_nonzero(z_contact_1) + np.count_nonzero(z_contact_2)
    
    return contact_voxels

def create_roi_mask(ref_img, target_img, roi_size, pixel_size, mask_pixel_value, img_size):
    """Creates a specified region of interest (ROI) mask using a reference image (or image stack) and
        applies it on the intended target image (or image stack).
    
    Args:
        ref_img(numpy.ndarray): Array defining reference image (or image stack). 
        target_img(numpy.ndarray): Array defining target image (or image stack). 
        roi_size(float): Size in micrometers for ROI.
        pixel_size(float): Image pixel size in micrometers.
        mask_pixel_value(int): Determines the value of the pixels on the mask.
        img_size(list[int]): List definifng image size width x height

  
    Returns: 
        numpy.ndarray: Array with target images after applying ROI mask.
    """
    
    binary_mask = ref_img > 0
    roi_mask = []
    mask_size = round(roi_size/pixel_size)
    distance_map = np.zeros((img_size[0], img_size[1]), dtype='uint16') # Todo: Change this to support images with different sizes
    for item in binary_mask:
        distance_map[:][:] = (sp.ndimage.distance_transform_edt(np.logical_not(item)))
        roi_mask.append((distance_map < mask_size))
    #print(len(aux_list))
    #roi_mask = np.zeros((625, 3000, 3000), dtype='uint16')
    #i = 0
    #for item in aux_list:
    #    roi_mask[i][:][:] = item
    #    i = i + 1
    #print(roi_mask.shape)
    #distance_map = sp.ndimage.distance_transform_edt(np.logical_not(binary_mask_1))  #computes the distance from background points to the nearest zero
    #binary_mask_2 = distance_map < mask_size

    target_img[~np.array(roi_mask, dtype='bool')] = mask_pixel_value
     
    return target_img

In [2]:
class Sample():  

    def __init__(self, name, num_weeks, ref_path, target_path, smooth_target, labels_to_smooth, pixel_size, label_dict, img_size):
        """Class defining a target sample
    
        Args:
            name(string): Name of the sample. 
            num_weeks(int): Define the number of the weeks of sample.
            ref_path(string): String containing path for reference images with proper file extension. 
                Example: "reference/*.tif" - e.g. the reference screw used for this sample       
            target_path(string): String containing path for target images (analysed sample) with proper file extension. 
                Example: "target/*.tif" - e.g. the labeled segmented images of the target sample
            pixel_size(float): Pixel size of images, in micrometers
            smooth_target(bool): When set to True, smooths labels indicated on labels_to_smooth on the target image
            labels_to_smooth(list[int]): List containing the labels which should be smoothed.
            label_dict(dict of string: int): Dictionary with the relation of label and correspoding pixel value
                Example: label_dict = {'background': 0, 'screw': 1, 'degradation_layer': 2, 'bone': 3}
            img_size(list[int]): List definifng image size width x height

        """
        
        self.name = name 
        self.num_weeks = num_weeks         
        self.ref_path = ref_path
        self.target_path = target_path
        self.pixel_size = pixel_size
        self.smooth_target = smooth_target
        self.labels_to_smooth = labels_to_smooth
        self.ref_image = (load_image(path=self.ref_path)).astype(np.int8)
        self.target_image = load_image(path=self.target_path, smooth=self.smooth_target, labels_to_smooth=self.labels_to_smooth)
        self.label_dict = label_dict    
        self.img_size = img_size
  
   
    def get_initial_volume(self): 
        """Gets initial sample volume in [mm^3]
    
        Returns: 
            float: Initial volume in [mm^3]  
        """
        
        to_mm = (self.pixel_size ** 3) / 1e09
        v_i = count_voxels_per_label(image=self.ref_image, label=self.label_dict['screw_ref']) * to_mm  
        
        return v_i

    def get_residual_volume(self): 
        """Gets residual sample volume in [mm^3]
    
        Returns: 
            float: Residual volume in [mm^3]  
        """
        
        to_mm = (self.pixel_size ** 3) / 1e09
        v_res = count_voxels_per_label(image=self.target_image, label=self.label_dict['screw']) * to_mm  

        return v_res   

    def get_initial_area(self): 
        """Gets initial sample area in [mm^2]
    
        Returns: 
            float: Initial area in [mm^2]  
        """
        
        to_mm = (self.pixel_size ** 2) / 1e06
        a_i = contact_area(image=self.ref_image, label1=self.label_dict['background'], label2=self.label_dict['screw_ref']) * to_mm
        
        return a_i 

    def get_time(self): 
        """Gets time of sample in [years]. Considers that a year is made of 48 weeks
    
        Returns: 
            float: Time of sample in [years]  
        """
        
        time = self.num_weeks / 48
        
        return time 

    def get_degradation_rate(self):
        """Calculates the degradation rate (DR) in [mm/year]

        Returns: 
            double: Degradation rate in [mm/year]
        """
        
        v_i = self.get_initial_volume()
        v_res = self.get_residual_volume()
        a_i = self.get_initial_area()
        print('v_i:', v_i)
        print('a_i:', a_i)
        print('v_res:', v_res)
        print('v_f:', (v_i - v_res))
        t = self.get_time()
        dr = (v_i - v_res) / (a_i * t)

        return dr
    
    def get_BIC_images(self, use_roi=True):
        """Gets the images containing labels for screw and screw+bone.
        
        Args:
            use_roi(bool): When set to True, chooses a region of interest of the obtained images.
        Returns: 
            numpy.ndarray: Array representing screw.
            numpy.ndarray: Array representing screw+bone.

        """       
        
        screw = filter_labels(image=self.target_image, labels=[self.label_dict['screw'], 
                                                               self.label_dict['degradation_layer']])
        screw[screw == self.label_dict['degradation_layer']] = self.label_dict['screw']

        screw_bone = filter_labels(image=self.target_image, labels=[self.label_dict['bone']])     
        screw_bone = screw_bone + screw
        
        if use_roi:
            screw = create_roi_mask(ref_img=self.ref_image,
                                    target_img=screw, 
                                    roi_size=1000, 
                                    pixel_size=self.pixel_size,
                                    mask_pixel_value=self.label_dict['background'],
                                    img_size=self.img_size)
            screw_bone = create_roi_mask(ref_img=self.ref_image,
                                         target_img=screw_bone, 
                                         roi_size=1000, 
                                         pixel_size=self.pixel_size,
                                         mask_pixel_value=self.label_dict['background'],
                                         img_size=self.img_size)
        
        return screw, screw_bone
    
    def get_bone_to_implant_contact(self):
        """Calculates the bone to implant contact (BIC)

        Returns: 
            double: Bone to implant contact 
        """
        
        screw, screw_bone = self.get_BIC_images(use_roi=True)
        contact_implant_bone = contact_area(image=screw_bone, 
                                            label1=self.label_dict['screw'], 
                                            label2=self.label_dict['bone'])
        contact_background_implant = contact_area(image=screw, 
                                                  label1=self.label_dict['screw'], 
                                                  label2=self.label_dict['background'])
        print('contact:', contact_implant_bone)
        print('residual:', contact_background_implant)
        bic = contact_implant_bone / contact_background_implant

        return bic

    def get_bone_volume_to_total_volume(self, roi_size):
        """Calculates the bone volume to total volume (BV/TV) 

        Args:
            roi_size(float): Size in micrometers for ROI

        Returns: 
            double: Bone volume to total volume 
        """
        
        masked_stack = create_roi_mask(ref_img=self.ref_image,
                                       target_img=self.target_image, 
                                       roi_size=roi_size, 
                                       pixel_size=self.pixel_size,
                                       mask_pixel_value=self.label_dict['screw'],
                                       img_size=self.img_size)
        histogram = skimage.exposure.histogram(masked_stack)
        v_bone = histogram[0][self.label_dict['bone']]
        v_roi = v_bone + histogram[0][self.label_dict['background']]
        print('bone:', v_bone)
        print('background:', (v_roi - v_bone))        
        bv_tv = v_bone / v_roi

        return bv_tv    

    def get_miou(ground_truth, prediction, labels):
        """Calculates mean intersection over union (mIoU) given a ground truth segmentation against a prediction.
        
        Args:
            ground_truth(numpy.ndarray): Array defining the ground truth image image (or image stack).
            prediction(numpy.ndarray): Array defining the predicted image (or image stack).
            labels(list[int]): list defining label. Ex.: [0, 1, 2, 3]

        Returns: 
            double: mIoU
        """

        iou_labels = jaccard_score(gt.flatten(),pred.flatten(), labels=labels, average=None)

        return np.mean(iou_labels)
    

In [6]:

# Load paths
ref_path = '/asap3/petra3/gpfs/p07/2020/data/11009431/processed/bric011_351_a/reco/enhanced_3000x3000x625/new_calculations/reference/*.tif'
target_path = '/asap3/petra3/gpfs/p07/2020/data/11009431/processed/bric010_342_c/reco/enhanced_3000x3000x625/new_calculations/segmented/*.tif'


In [7]:
# Inputs
sample = Sample(name='bric011_351_a',
                num_weeks=12, 
                ref_path=ref_path, 
                target_path=target_path, 
                smooth_target=False, 
                labels_to_smooth=[1,2],                
                pixel_size=3.8, 
                label_dict={'background': 0,
                            'screw': 1,
                            'screw_ref': 2,
                            'degradation_layer': 2,
                            'bone': 3},
               img_size = [3000, 3000]) 

In [8]:
import time

start_time = time.time()

#print('Training name:', '3D_nnUNet_bric011_351_a')
print('Sample name:', sample.name)
print('Sample time:', sample.num_weeks, 'weeks')
print('Sample DR:', sample.get_degradation_rate())
print('Sample BIC:', sample.get_bone_to_implant_contact())
print('Sample BV/TV:', sample.get_bone_volume_to_total_volume(roi_size=1000))
print("--- %s seconds ---" % (time.time() - start_time))

Sample name: bric011_351_a
Sample time: 12 weeks
(array([4405528510,          0,  328471490]), array([0, 1, 2]))
(array([2977641387,  267783033,   33623654, 1454951926]), array([0, 1, 2, 3]))
v_i: 18.02388759928
a_i: 50.64200416
v_res: 14.693790586775997
v_f: 3.3300970125040017
Sample DR: 0.2630304284153356


KeyboardInterrupt: 